In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch
from tqdm.auto import tqdm
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch import optim
plt.rcParams['font.family'] = 'Malgun Gothic'
# 데이터 불러오기
df_all2=pd.read_csv('../data_v3/감성점수와 일별뉴스 합친거3.csv',encoding='utf-8')

In [120]:
df_all2

,전체_date,판정일,경락단가,경락단가_평균,datetime,new_content_x,day_sentiment
0,2018-01-02,2018-01-02,3750.366093,3750.366093,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000
1,2018-01-03,2018-01-03,3547.574932,3547.574932,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000
2,2018-01-04,2018-01-04,3696.060897,3696.060897,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167
3,2018-01-05,2018-01-05,3770.156364,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167
4,2018-01-06,2018-01-05,3770.156364,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167
...,...,...,...,...,...,...,...
1455,2021-12-27,2021-12-27,4547.071429,4547.071429,2021-12-27,\n▲ 연암대가 최근 ICT 융복합 스마트 양돈실습장 구축을 완료했다.\n \n연암...,0.175000
1456,2021-12-28,2021-12-28,4373.212544,4373.212544,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270
1457,2021-12-29,2021-12-29,4278.649485,4278.649485,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270
1458,2021-12-30,2021-12-30,4322.080986,4322.080986,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270


In [121]:
df_all=pd.read_excel('../data_v3/전국경락단가.xlsx')

df_all

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  """Entry point for launching an IPython kernel.


,date,price
0,2018-01-02,4104
1,2018-01-03,4049
2,2018-01-04,3920
3,2018-01-05,3917
4,2018-01-08,4090
...,...,...
999,2021-12-27,4825
1000,2021-12-28,4674
1001,2021-12-29,4904
1002,2021-12-30,5000


In [122]:
df_all['date']= pd.to_datetime(df_all['date'])
df_all2['전체_date']= pd.to_datetime(df_all2['전체_date'])

In [123]:
merged_df = df_all2.merge(df_all, left_on='전체_date', right_on='date', how='left')

In [124]:
merged_df.isnull().sum()

전체_date            0
판정일                0
경락단가               0
경락단가_평균            0
datetime           0
new_content_x      0
day_sentiment      0
date             456
price            456
dtype: int64

In [125]:
#결측치 제거
# 누락된 값을 "ffill"로 채우기
merged_df = merged_df.fillna(method='ffill')

In [126]:
merged_df

,전체_date,판정일,경락단가,경락단가_평균,datetime,new_content_x,day_sentiment,date,price
0,2018-01-02,2018-01-02,3750.366093,3750.366093,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000,2018-01-02,4104.0
1,2018-01-03,2018-01-03,3547.574932,3547.574932,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000,2018-01-03,4049.0
2,2018-01-04,2018-01-04,3696.060897,3696.060897,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167,2018-01-04,3920.0
3,2018-01-05,2018-01-05,3770.156364,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167,2018-01-05,3917.0
4,2018-01-06,2018-01-05,3770.156364,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167,2018-01-05,3917.0
...,...,...,...,...,...,...,...,...,...
1455,2021-12-27,2021-12-27,4547.071429,4547.071429,2021-12-27,\n▲ 연암대가 최근 ICT 융복합 스마트 양돈실습장 구축을 완료했다.\n \n연암...,0.175000,2021-12-27,4825.0
1456,2021-12-28,2021-12-28,4373.212544,4373.212544,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270,2021-12-28,4674.0
1457,2021-12-29,2021-12-29,4278.649485,4278.649485,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270,2021-12-29,4904.0
1458,2021-12-30,2021-12-30,4322.080986,4322.080986,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270,2021-12-30,5000.0


In [127]:
merged_df.columns=['전체_date','경락단가_date','경락단가','경락단가_평균','뉴스_date','new_content_sum','day_sentiment','도매가격_date','도매가격']

In [128]:
merged_df=merged_df[['전체_date','경락단가_date','경락단가','뉴스_date','new_content_sum','day_sentiment','도매가격_date','도매가격']]

In [129]:
merged_df

,전체_date,경락단가_date,경락단가,뉴스_date,new_content_sum,day_sentiment,도매가격_date,도매가격
0,2018-01-02,2018-01-02,3750.366093,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000,2018-01-02,4104.0
1,2018-01-03,2018-01-03,3547.574932,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000,2018-01-03,4049.0
2,2018-01-04,2018-01-04,3696.060897,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167,2018-01-04,3920.0
3,2018-01-05,2018-01-05,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167,2018-01-05,3917.0
4,2018-01-06,2018-01-05,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167,2018-01-05,3917.0
...,...,...,...,...,...,...,...,...
1455,2021-12-27,2021-12-27,4547.071429,2021-12-27,\n▲ 연암대가 최근 ICT 융복합 스마트 양돈실습장 구축을 완료했다.\n \n연암...,0.175000,2021-12-27,4825.0
1456,2021-12-28,2021-12-28,4373.212544,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270,2021-12-28,4674.0
1457,2021-12-29,2021-12-29,4278.649485,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270,2021-12-29,4904.0
1458,2021-12-30,2021-12-30,4322.080986,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270,2021-12-30,5000.0


In [130]:
merged_df=merged_df[['전체_date','도매가격_date','도매가격','경락단가_date','경락단가','뉴스_date','new_content_sum','day_sentiment']]

In [131]:
merged_df

,전체_date,도매가격_date,도매가격,경락단가_date,경락단가,뉴스_date,new_content_sum,day_sentiment
0,2018-01-02,2018-01-02,4104.0,2018-01-02,3750.366093,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000
1,2018-01-03,2018-01-03,4049.0,2018-01-03,3547.574932,2018-01-02,지난해 11월 양돈용 배합사료 생산량이 크게 증가했다.농림축산식품부에 따르면 지난...,0.000000
2,2018-01-04,2018-01-04,3920.0,2018-01-04,3696.060897,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167
3,2018-01-05,2018-01-05,3917.0,2018-01-05,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167
4,2018-01-06,2018-01-05,3917.0,2018-01-05,3770.156364,2018-01-04,"\t\t\t\t\t종돈 4,409마리 수입…5년만에 최고\r\n종개협 신규 종돈장...",-0.104167
...,...,...,...,...,...,...,...,...
1455,2021-12-27,2021-12-27,4825.0,2021-12-27,4547.071429,2021-12-27,\n▲ 연암대가 최근 ICT 융복합 스마트 양돈실습장 구축을 완료했다.\n \n연암...,0.175000
1456,2021-12-28,2021-12-28,4674.0,2021-12-28,4373.212544,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270
1457,2021-12-29,2021-12-29,4904.0,2021-12-29,4278.649485,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270
1458,2021-12-30,2021-12-30,5000.0,2021-12-30,4322.080986,2021-12-28,범 양돈산업계를 아우르는 컨트롤타워 출현이 가시화되고 있다.대한한돈협회 손세희 회장...,-0.116270


In [134]:
merged_df.to_csv('../data_v3/(최종V1)도매가격 감성점수 경락단가 merge.csv',encoding='utf-8',index=False)

In [61]:
# 검색할 년, 월, 일 설정
'''year_to_search = 2018
month_to_search = 3
day_to_search = 20


filtered_data = df_all2[(df_all2['판정일'].dt.year == year_to_search) &
                   (df_all2['판정일'].dt.month == month_to_search) &
                   (df_all2['판정일'].dt.day == day_to_search)]'''

#filtered_data

In [117]:
merged_df.describe()

,도매가격,경락단가,day_sentiment
count,1460.000000,1460.000000,1460.000000
mean,4582.154110,4361.188197,-0.122471
std,798.459952,787.475009,0.253576
min,2021.000000,2120.549342,-1.000000
25%,3972.000000,3809.038049,-0.267986
50%,4573.000000,4322.888931,-0.093750
75%,5151.000000,4991.123099,0.000000
max,6649.000000,6647.612500,1.000000
